# 学習データ生成ブック

In [3]:
import sys

sys.path.append("../")

In [4]:
from utils.wikipedia_category import WikipediaCategory

wikipedia_category = WikipediaCategory()

STEM_WEIGHTS = wikipedia_category.CATEGORY_WEIGHTS
STEM = wikipedia_category.CATEGORY
EXCLUDE_CATEGORIES = wikipedia_category.EXCLUDE_CATEGORIES

プロンプトの指定

In [5]:
delimiter = "####"
system_message = f"""
You will be provided with text from Wikipedia, separated by {delimiter} characters. 
From that text, create only one multiple-choice question with 5 options, and express it as a dictionary object. Furthermore, store that dictionary object in a Python list and output it. The dictionary object follows this format:
'question': <question based on the TEXT>
'option_1': <question answer option>
'option_2': <question answer option>
'option_3': <question answer option>
'option_4': <question answer option>
'option_5': <question answer option>
'answer': <answer option key label>
Among the suggested options, the correct one is indicated by assigning the key label ('option_1', 'option_2', 'option_3', 'option_4', 'option_5') to the 'answer' field.
The question, answer, and question options should be broad, challenging, detailed, and based on the given text, and should not be easy.
Please output only the list of objects, without including anything else.
"""

In [6]:
from utils.wikipedia_question_generator import WikipediaQuestionGenerator

options_set = wikipedia_category.OPTIONS_SET
response_keys_set = wikipedia_category.RESPONSE_KEYS_SET
wikipedia_question_generator = WikipediaQuestionGenerator(STEM,STEM_WEIGHTS,EXCLUDE_CATEGORIES,system_message,options_set,response_keys_set)

In [7]:
multiple_choice_questions, seen_pages, attempts_list \
    = wikipedia_question_generator.gather_multiple_choice_question_dataset(1,10,[],seed=13630338)

In [ ]:
df_mcq = WikipediaQuestionGenerator.conver_to_compet_format_df(multiple_choice_questions)

# dumpしたデータセットを復元する場合

In [1]:
import pickle

with open('../output/pickle/generate_question_2023_09_14.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [2]:
import pandas as pd

dataset_df = pd.DataFrame(dataset)

In [37]:
dataset_df.rename(
            columns = {
                'question': 'prompt', 
                'option_1': 'A', 
                'option_2': 'B', 
                'option_3': 'C', 
                'option_4': 'D', 
                'option_5': 'E'
            }, 
            inplace = True
        )

answer_subjects = {
            'option_1': 'A', 
            'option_2': 'B', 
            'option_3': 'C', 
            'option_4': 'D', 
            'option_5': 'E'
        }
dataset_df["answer"] = dataset_df["answer"].map(answer_subjects)

dataset_df.insert(0, "id", list(range(len(dataset_df))))

df_compet = dataset_df.drop(columns=["wiki_text", "page_id", "page_title", "category_label"])

In [38]:
from utils import get_str_today

df_compet.to_csv(f"../output/dataset/stem_dataset_{get_str_today()}.csv", index=False)

# dumpしたseen_pages(これまでの作問に使ったページ)

In [1]:
import pickle

with open('../output/pickle/seen_pages_2023_09_12.pickle', 'rb') as f:
    seen_pages = pickle.load(f)

In [2]:
seen_pages

[5944849, 10977940, 23376109]

# 学習の生成を再現する場合

再現したい状況までのseen_pagesとseedを引数にすることで、途中から再実行することができる

In [7]:
multiple_choice_questions, seen_pages, attempts_list= wikipedia_question_generator.gather_multiple_choice_question_dataset(pages_count=1,seen_pages=seen_pages,seed=4)

In [8]:
df_mcq = WikipediaQuestionGenerator.conver_to_compet_format_df(multiple_choice_questions)